In [1]:
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite
import numpy as np

print(sklearn.__version__)

0.18.1


# Let's use CoNLL 2002 data to build a NER system

CoNLL2002 corpus is available in NLTK. We use Spanish data.

In [2]:
nltk.corpus.conll2002.fileids()

['esp.testa', 'esp.testb', 'esp.train', 'ned.testa', 'ned.testb', 'ned.train']

In [3]:
%%time
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))

CPU times: user 3.12 s, sys: 76 ms, total: 3.2 s
Wall time: 3.19 s


Data format:

In [4]:
train_sents[0]

[('Melbourne', 'NP', 'B-LOC'),
 ('(', 'Fpa', 'O'),
 ('Australia', 'NP', 'B-LOC'),
 (')', 'Fpt', 'O'),
 (',', 'Fc', 'O'),
 ('25', 'Z', 'O'),
 ('may', 'NC', 'O'),
 ('(', 'Fpa', 'O'),
 ('EFE', 'NC', 'B-ORG'),
 (')', 'Fpt', 'O'),
 ('.', 'Fp', 'O')]

## Features

Next, define some features. In this example we use word identity, word suffix, word shape and word POS tag; also, some information from nearby words is used. 

This makes a simple baseline, but you certainly can add and remove some features to get (much?) better results - experiment with it.

In [5]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag,
        'postag[:2]=' + postag[:2]
    ]
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + postag1,
            '-1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('BOS')

    if i > 1:
        word2 = sent[i-2][0]
        postag2 = sent[i-2][1]
        features.extend([
            '-2:word.lower=' + word2.lower(),
            '-2:word.istitle=%s' % word2.istitle(),
            '-2:word.isupper=%s' % word2.isupper(),
            '-2:postag=' + postag2,
            '-2:postag[:2]=' + postag2[:2],
            ])

    if i > 2:
        word3 = sent[i-2][0]
        postag3 = sent[i-2][1]
        features.extend([
            '-3:word.lower=' + word3.lower(),
            '-3:word.istitle=%s' % word3.istitle(),
            '-3:word.isupper=%s' % word3.isupper(),
            '-3:postag=' + postag3,
            '-3:postag[:2]=' + postag3[:2],
            ])

    if i < len(sent)-3:
        word3 = sent[i+3][0]
        postag3 = sent[i+3][1]
        features.extend([
            '+3:word.lower=' + word3.lower(),
            '+3:word.istitle=%s' % word3.istitle(),
            '+3:word.isupper=%s' % word3.isupper(),
            '+3:postag=' + postag3,
            '+3:postag[:2]=' + postag3[:2],
            ])
        
    if i < len(sent)-2:
        word2 = sent[i+2][0]
        postag2 = sent[i+2][1]
        features.extend([
            '+2:word.lower=' + word2.lower(),
            '+2:word.istitle=%s' % word2.istitle(),
            '+2:word.isupper=%s' % word2.isupper(),
            '+2:postag=' + postag2,
            '+2:postag[:2]=' + postag2[:2],
        ])

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:postag=' + postag1,
            '+1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('EOS')
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]    

This is what word2features extracts:

In [6]:
sent2features(train_sents[0])[0]

['bias',
 'word.lower=melbourne',
 'word[-3:]=rne',
 'word[-2:]=ne',
 'word.isupper=False',
 'word.istitle=True',
 'word.isdigit=False',
 'postag=NP',
 'postag[:2]=NP',
 'BOS',
 '+3:word.lower=)',
 '+3:word.istitle=False',
 '+3:word.isupper=False',
 '+3:postag=Fpt',
 '+3:postag[:2]=Fp',
 '+2:word.lower=australia',
 '+2:word.istitle=True',
 '+2:word.isupper=False',
 '+2:postag=NP',
 '+2:postag[:2]=NP',
 '+1:word.lower=(',
 '+1:word.istitle=False',
 '+1:word.isupper=False',
 '+1:postag=Fpa',
 '+1:postag[:2]=Fp']

Extract the features from the data:

In [7]:
%%time
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]
X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

CPU times: user 5.8 s, sys: 224 ms, total: 6.03 s
Wall time: 6.02 s


## Train the model

To train the model, we create pycrfsuite.Trainer, load the training data and call 'train' method. 
First, create pycrfsuite.Trainer and load the training data to CRFsuite:

In [8]:
%%time
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

CPU times: user 7.04 s, sys: 28 ms, total: 7.07 s
Wall time: 7.08 s


In [9]:
def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

Set training parameters. We will use L-BFGS training algorithm (it is default) with Elastic Net (L1 + L2) regularization.

In [ ]:
c1s = np.logspace(-3, 0, 4)
c2s = np.logspace(-3, -1, 3)

for c1 in c1s:
    for c2 in c2s:
        trainer.set_params({
            'c1': c1,   # coefficient for L1 penalty
            'c2': c2,  # coefficient for L2 penalty
            'max_iterations': 50,  # stop earlier

            # include transitions that are possible, but not observed
            'feature.possible_transitions': True
        })

        trainer.train('conll2002-esp.crfsuite')
        
        tagger = pycrfsuite.Tagger()
        tagger.open('conll2002-esp.crfsuite')
        
        y_pred = [tagger.tag(xseq) for xseq in X_test]
        print('c1=', c1, ' c2=',  c2)
        print(bio_classification_report(y_test, y_pred))
    

c1= 0.001  c2= 0.001
             precision    recall  f1-score   support

      B-LOC       0.83      0.76      0.79      1084
      I-LOC       0.75      0.59      0.66       325
     B-MISC       0.71      0.50      0.58       339
     I-MISC       0.80      0.54      0.65       557
      B-ORG       0.79      0.86      0.83      1400
      I-ORG       0.85      0.81      0.83      1104
      B-PER       0.89      0.86      0.87       735
      I-PER       0.91      0.92      0.92       634

avg / total       0.83      0.78      0.80      6178

c1= 0.001  c2= 0.01
             precision    recall  f1-score   support

      B-LOC       0.83      0.77      0.80      1084
      I-LOC       0.74      0.62      0.67       325
     B-MISC       0.69      0.51      0.59       339
     I-MISC       0.75      0.56      0.64       557
      B-ORG       0.80      0.86      0.83      1400
      I-ORG       0.84      0.84      0.84      1104
      B-PER       0.88      0.87      0.88       735
 

Choose the best parameters from previous cell

In [12]:
c1 = 0.1
c2 = 0.001

trainer.set_params({'c1': c1, 'c2': c2, 'max_iterations': 50,
    'feature.possible_transitions': True})

Train the model:

In [13]:
%%time
trainer.train('conll2002-esp.crfsuite')

CPU times: user 1min 40s, sys: 72 ms, total: 1min 40s
Wall time: 1min 40s


trainer.train saves model to a file:

In [ ]:
!ls -lh ./conll2002-esp.crfsuite

We can also get information about the final state of the model by looking at the trainer's logparser. If we had tagged our input data using the optional group argument in add, and had used the optional holdout argument during train, there would be information about the trainer's performance on the holdout set as well. 

In [ ]:
trainer.logparser.last_iteration

## Make predictions

To use the trained model, create pycrfsuite.Tagger, open the model and use "tag" method:

In [ ]:
tagger = pycrfsuite.Tagger()
tagger.open('conll2002-esp.crfsuite')

Let's tag a sentence to see how it works:

In [ ]:
example_sent = test_sents[0]
print(' '.join(sent2tokens(example_sent)), end='\n\n')

print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
print("Correct:  ", ' '.join(sent2labels(example_sent)))

## Evaluate the model

Predict entity labels for all sentences in our testing set ('testb' Spanish data):

In [ ]:
%%time
y_pred = [tagger.tag(xseq) for xseq in X_test]

..and check the result. Note this report is not comparable to results in CONLL2002 papers because here we check per-token results (not per-entity). Per-entity numbers will be worse.  

In [ ]:
print(bio_classification_report(y_test, y_pred))

## Let's check what classifier learned

In [ ]:
from collections import Counter
info = tagger.info()

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(info.transitions).most_common(15))

print("\nTop unlikely transitions:")
print_transitions(Counter(info.transitions).most_common()[-15:])

We can see that, for example, it is very likely that the beginning of an organization name (B-ORG) will be followed by a token inside organization name (I-ORG), but transitions to I-ORG from tokens with other labels are penalized. Also note I-PER -> B-LOC transition: a positive weight means that model thinks that a person name is often followed by a location.

Check the state features:

In [ ]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-6s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(info.state_features).most_common(20))

print("\nTop negative:")
print_state_features(Counter(info.state_features).most_common()[-20:])

In [ ]:
# Counter(info.state_features)

Some observations:

* **8.743642 B-ORG  word.lower=psoe-progresistas** - the model remembered names of some entities - maybe it is overfit, or maybe our features are not adequate, or maybe remembering is indeed helpful;
* **5.195429 I-LOC  -1:word.lower=calle**: "calle" is a street in Spanish; model learns that if a previous word was "calle" then the token is likely a part of location;
* **-3.529449 O      word.isupper=True**, ** -2.913103 O      word.istitle=True **: UPPERCASED or TitleCased words are likely entities of some kind;
* **-2.585756 O      postag=NP** - proper nouns (NP is a proper noun in the Spanish tagset) are often entities.

## What to do next

1. Load 'testa' Spanish data.
2. Use it to develop better features and to find best model parameters.
3. Apply the model to 'testb' data again.

The model in this notebook is just a starting point; you certainly can do better!